## Load in and clean data before we start modelling it

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
#load  in data and just consider columsn of importance for analysis
df_task=pd.read_csv('task.csv')
df_task_r=df_task[['Participant Public ID','display','forced_choice','Response','image2','test_image1', 'test_image2','test_image1_value', 
                   'test_image2_value','image_query2', 'image_query1']]
df_task_r=df_task_r.replace('response_text_entry','query_internal_probability') 

# Define best-action dictionary for present design
# best action per condition (6 conditions)
best_answer_key={'rare_threat_1': [[0, 'Pinecone 1.jpg'], [1, 'Pumpkin 1.jpg']], 
                 'rare_threat_2': [[0, 'Keyboard 3.jpg'], [1, 'Office supplies 2.jpg']],
                 'common_threat_1': [[0, 'Fire hydrant 1.jpg'], [1, 'Fence 2.jpg']],
                 'common_threat_2': [[0, 'Bricks 1.jpg'], [1, 'Barrels 1.jpg']],
                 'neutral_1': [[0, 'Snow 3.jpg'], [1, 'Skyscraper 1.jpg']],
                 'neutral_2': [[0, 'Clean 1.jpg'], [1, 'Cotton swabs 3.jpg']]}

#best answers per condition: lists
rt1=[]
rt2=[]
ct1=[]
ct2=[]
n1=[]
n2=[]

tally=0
invalid_scores={'NaN'}
counter=0
conditions=[]
start_new_test_set=0
start_new_subject=0
best_action_tally=0
condition_counter=0
sub_counter=0
current_subject=1

#subject specific data
rt_sub=[]
ct_sub=[]
neut_sub=[]
current_choice_data=[]
choice_data_3d=np.zeros((13,3,40)) #to be populated below

for row,data in df_task_r.iterrows():
    
    if str(df_task_r['display'][row]).startswith('test'):
        
        if counter==0:
            conditions.append(df_task_r['display'][row][5:])
            condition_info=best_answer_key[conditions[counter]]
            new_condition=0
            counter+=1
            

        elif df_task_r['display'][row][5:]!=conditions[counter-1]:
                    conditions.append(df_task_r['display'][row][5:])
                    condition_info=best_answer_key[conditions[counter]]
                    if conditions[counter-1]=='rare_threat_1':
                        rt1.append(best_action_tally)
                        rt_sub.append(current_choice_data)

                    elif conditions[counter-1]=='rare_threat_2':
                        rt2.append(best_action_tally)
                        rt_sub.append(current_choice_data)

                    elif conditions[counter-1]=='common_threat_1':
                        ct1.append(best_action_tally)
                        ct_sub.append(current_choice_data)

                    elif conditions[counter-1]=='common_threat_2':
                        ct2.append(best_action_tally)
                        ct_sub.append(current_choice_data)

                    elif conditions[counter-1]=='neutral_1':
                        n1.append(best_action_tally)
                        neut_sub.append(current_choice_data)

                    elif conditions[counter-1]=='neutral_2':
                        n2.append(best_action_tally)
                        neut_sub.append(current_choice_data)
                    counter+=1
                    best_action_tally=0
                    current_choice_data=[]
                    condition_counter+=1
                    #after 6 blocks, new subject
                    if condition_counter>5:
                        current_subject+=1
                        neut_sub=neut_sub[0]+neut_sub[1]
                        if len(neut_sub)<40:
                            neut_sub=[int(x) for x in neut_sub+np.zeros(40-len(neut_sub)).tolist()]
             
                            
                        rt_sub=rt_sub[0]+rt_sub[1]
                        if len(rt_sub)<40:
                            rt_sub=[int(x) for x in rt_sub+np.zeros(40-len(rt_sub)).tolist()]
                      
                        ct_sub=ct_sub[0]+ct_sub[1]
                        if len(ct_sub)<40:
                            ct_sub=[int(x) for x in ct_sub+np.zeros(40-len(ct_sub)).tolist()]
                        
                        choice_data_3d[sub_counter,0]=neut_sub
                        choice_data_3d[sub_counter,1]=ct_sub
                        choice_data_3d[sub_counter,2]=rt_sub
                        sub_counter+=1                                               
                        condition_counter=0
                        rt_sub=[]
                        ct_sub=[]
                        neut_sub=[]
        
        else:
            new_condition=0
                    
        #Get values and convert from strings to floating point
        value1=df_task_r['test_image1_value'][row]
        if "p" in value1:
            value1=float(value1[0:2])*0.01
        else:
            value1=float(value1[1])
        value2=df_task_r['test_image2_value'][row]
        if "p" in value2:
            value2=float(value2[0:2])*0.01
        else:
            value2=float(value2[1])
        
        if value1>value2:
            best_option=df_task_r['test_image1'][row]
        else:
            best_option=df_task_r['test_image2'][row]
        
            
        #get response and convert to integer
        try:
            current_response=int(df_task_r['Response'][row])
        except:
            current_response='missing'
                
            
        
        
        # determine if participant made best choice
        for info_total in condition_info:
            for info in info_total:
                if best_option == str(info):
                    best_action=info_total[0]

        #for last subject only that doesn't meet the condition above for indexing     
        if row==17254:
            ct2.append(best_action_tally)
            ct_sub.append(current_choice_data)
            neut_sub=neut_sub[0]+neut_sub[1]
            if len(neut_sub)<40:
                neut_sub=[int(x) for x in neut_sub+np.zeros(40-len(neut_sub)).tolist()]


            rt_sub=rt_sub[0]+rt_sub[1]
            if len(rt_sub)<40:
                rt_sub=[int(x) for x in rt_sub+np.zeros(40-len(rt_sub)).tolist()]

            ct_sub=ct_sub[0]+ct_sub[1]
            if len(ct_sub)<40:
                ct_sub=[int(x) for x in ct_sub+np.zeros(40-len(ct_sub)).tolist()]

            choice_data_3d[sub_counter,0]=neut_sub
            choice_data_3d[sub_counter,1]=ct_sub
            choice_data_3d[sub_counter,2]=rt_sub
            sub_counter+=1                                               
            condition_counter=0
            rt_sub=[]
            ct_sub=[]
            neut_sub=[]

        else:
            if current_response==best_action:
                best_action_tally+=1
                current_choice_data.append(1.0)
            elif current_response=='missing':
                x='missing'
            else:
                current_choice_data.append(0.0)
                

#convert to numpy arrays    
rt1=np.array(rt1)
rt2=np.array(rt2)
ct1=np.array(ct1)
ct2=np.array(ct2)
n1=np.array(n1)
n2=np.array(n2)

choice_data_3d = choice_data_3d.astype(int)

/home/paulsharp/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##  Modelling pilot data hierarchically: explaining choice data

In the present experiment, participants chose an action to obtain a reward based off a latent transition matrix they've presumably learned. The hypothesis is that this transition matrix is altered due to experimentally-manipulated features (valence of emotional distractors during learning) and person-specific factors that are not manipulated (level of chronic worry). 

We can conceive of the generative model of my data in the following way. We'll start from the bottom up. Each individual's decision is either a 1 or 0 (did they select the best-available option or not). The best-available option is the action that maximizes the EV according to a greedy policy (which is normative here given that learning has terminated once decision-making begins; that is, if I've learned I have a 60% chance pressing X will get me to the highest reward, and Y will get me there 40% of the time, I should always choose X). 

The $\theta$ parameter determines a subjects' choice. If their bias is 0, it is an index that they're always choosing the worst action. If the bias is 1, they're maximizing performance. 

We use a Bernoulli likelihood to define the **subject-specific** data-generating process to explain their choice data:

$Choice_{i,t}$ $\sim$ Bernoulli$(\theta_{i,k})$ Indices: i=subjects, t=trials, k=condition, where k(1) = neutral, k(2) = positive and k(3) = negative. 

These decicision biases, $\theta$, vary by subject and condition. Each subject has a parameter that encodes the effect of each experimental condition $\gamma_{k}$. A logistic link function is necessary to take a set spanning all values to the 0 to 1 range in order to be appropriate for the Bernoulli likelihood.

$\theta_{i,k}\,= \begin{cases}
    \text{logistic}\, (\gamma_{i,1}),& \text{if } k=1\\
    \text{logistic}\,(\gamma_{i,2}),& \text{if } k=2\\
    \text{logistic}\,(\gamma_{i,3}),& \text{if } k=3\\
\end{cases}
$

$\gamma_{1}$, $\gamma_{2}$, and $\gamma_{3}$ represent the biases for **each condition** per subject, where each is drawn from **group-level distribution over condition effects**:

$\gamma_{i,1}$ $\sim \mathcal{N}(\mu_{1},\sigma_{1})$

$\gamma_{i,2}$ $\sim \mathcal{N}(\mu_{2},\sigma_{2})$

$\gamma_{i,3}$ $\sim \mathcal{N}(\mu_{3},\sigma_{3})$

Because we do not know *a priori* what the population-level distribution over each effect should be, we also need a prior distribution on these parameters. Priors over **experimental effects** at the population level:

Mean Prior:
$\mathcal{N}(\mathcal{M_{condition}},\sigma_{condition})$
Variance Prior:
$\mathcal{Gamma}(\mathcal{S_{condition}},\mathcal{K_{condition}})$

The posterior joint distribution one is trying to estimate is: $p(\gamma_{1,1}...\gamma_{subject_i,condition_k},\mu_{k},\sigma_{k}|data)$. Thus we must estimate $(i \cdot k) + 6_{groupLevel}$ parameters, which for the present dataset, is a 45-dimensional distribution.We use pyStan to fit the model to the data below.

## Build hierarchical model in pyStan

In [3]:
from pystan import StanModel


model_input='''
data {
    int<lower=0> N;      // # of subjects
    int N_cond;
    int T_max;  // max # of trials across subjects
    int Choice[N, N_cond, T_max]; // Choices for each subject, condition, and trial
}

parameters {  
 
  // Parameters for group-level parameters
  real mu_neutral;
  real mu_positive;
  real mu_negative;
  
  real<lower=0> sigma_neutral;
  real<lower=0> sigma_positive;  
  real<lower=0> sigma_negative;
  
  
  // Individual-level parameters
  real gamma_neutral[N];
  real gamma_positive[N];
  real gamma_negative[N];
}

transformed parameters {
  vector[N] theta_neutral;
  vector[N] theta_positive;
  vector[N] theta_negative;

  // For all subjects, incorporate baseline and experimental effect, 
  // then convert to 0-1 scale via logistic function. 
  
  for (i in 1:N) {
    theta_neutral[i] = inv_logit(gamma_neutral[i]);
    theta_positive[i] = inv_logit(gamma_positive[i]);
    theta_negative[i] = inv_logit(gamma_negative[i]);
    }
}


model {
  // Priors on group-level effects
  mu_neutral   ~ normal(0,10);
  mu_positive ~ normal(0,10);
  mu_negative ~ normal(0,10);
  
  sigma_neutral ~ gamma(1,1);
  sigma_positive ~ gamma(1,1);
  sigma_negative ~ gamma(1,1);

  
  // Priors on individual parameters
  gamma_neutral ~ normal(mu_neutral,sigma_neutral);
  gamma_positive ~ normal(mu_positive,sigma_positive);
  gamma_negative ~ normal(mu_negative,sigma_negative);

    
  // Generate data for each subject via Bernoulli likelihood
  for (i in 1:N) {
    // Neutral condition choices
    Choice[i,1,:] ~ bernoulli(theta_neutral[i]);
    // Positive condition choices
    Choice[i,2,:] ~ bernoulli(theta_positive[i]);
    // Negative condition choices
    Choice[i,3,:] ~ bernoulli(theta_negative[i]);    
  }
}

'''
data_input = {'N': 13, #subjects
                     'N_cond': 3, # conditions
                     'T_max': 40, # trials per condition
                     'Choice':choice_data_3d #choice data in a 3d Vector
                    }

controls={}
controls['adapt_delta']=0.8

model_fit = StanModel(model_code=model_input)
fit = model_fit.sampling(data=data_input,iter=4000,control=controls)
print(fit)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_26e96afb883e887181defe52649573b7 NOW.


Inference for Stan model: anon_model_26e96afb883e887181defe52649573b7.
4 chains, each with iter=4000; warmup=2000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=8000.

                     mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu_neutral           3.93    0.02    1.3   1.76   3.05   3.78   4.65   6.96   2788    1.0
mu_positive           1.9    0.02   1.35  -0.62   1.01   1.85   2.74   4.71   7259    1.0
mu_negative          1.46    0.01   0.92  -0.29   0.86   1.43   2.03   3.37   7151    1.0
sigma_neutral        3.58    0.02   1.01   2.08   2.87   3.42   4.11   6.05   2089    1.0
sigma_positive       4.36    0.02   1.02   2.77   3.63   4.23   4.94   6.76   3855    1.0
sigma_negative       3.11    0.01   0.72    2.0    2.6    3.0   3.52   4.76   4332    1.0
gamma_neutral[1]     0.24  2.9e-3   0.32  -0.37   0.02   0.24   0.46   0.89  12233    1.0
gamma_neutral[2]     6.82    0.05   2.66   3.16   4.88   6.28   8.18  13.35   2988    1.0
g